In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from PIL import Image
import pandas as pd
import cv2
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.optimizers import RMSprop ,Adadelta ,SGD ,Adamax

### Reading the CSV file and the images

In [12]:
# Read the CSV file and create a dataframe
main_csv_file = "data_labels_mainData.csv"
extra_csv_file ="data_labels_extraData.csv"
data = pd.read_csv(main_csv_file)
extra_data = pd.read_csv(extra_csv_file)
# Read the images from the directory and convert them to arrays
image_dir = "patch_images"
image_arrays = []
for image_name in data["ImageName"]:
    img = cv2.imread(os.path.join(image_dir, image_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (32, 32))
    image_arrays.append(img)
image_arrays = np.array(image_arrays)

#Merge the image arrays with the CSV dataframe
data["image_data"] = list(image_arrays)
# data = data.drop("ImageName", axis=1)
data

,InstanceID,patientID,ImageName,cellTypeName,cellType,isCancerous,image_data
0,22405,1,22405.png,fibroblast,0,0,"[[[242, 213, 237], [239, 210, 234], [236, 206,..."
1,22406,1,22406.png,fibroblast,0,0,"[[[221, 175, 211], [223, 175, 211], [229, 180,..."
2,22407,1,22407.png,fibroblast,0,0,"[[[247, 243, 247], [248, 242, 246], [247, 240,..."
3,22408,1,22408.png,fibroblast,0,0,"[[[243, 242, 242], [243, 241, 241], [243, 241,..."
4,22409,1,22409.png,fibroblast,0,0,"[[[189, 130, 189], [187, 127, 185], [187, 126,..."
...,...,...,...,...,...,...,...
9891,1625,60,1625.png,epithelial,2,1,"[[[171, 138, 180], [206, 175, 217], [238, 209,..."
9892,1626,60,1626.png,epithelial,2,1,"[[[212, 173, 228], [191, 151, 212], [172, 132,..."
9893,1627,60,1627.png,epithelial,2,1,"[[[175, 141, 193], [191, 162, 205], [207, 180,..."
9894,1628,60,1628.png,epithelial,2,1,"[[[227, 202, 232], [188, 159, 202], [180, 146,..."


### Function to get the accuracy on the test data and get prediction on the extra dataset

In [3]:
def predict_extra_data(model):
    image_names = extra_data["ImageName"]
    for i in range(10):
        image_loc = "patch_images/"+ str(image_names[i])
        new_image = cv2.imread(image_loc)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
        new_image = cv2.resize(new_image, (32, 32))
        new_image = np.array(new_image)
        new_image = np.expand_dims(new_image, axis=0)
        prediction = model.predict(new_image)[0][0]
        if prediction >= 0.5:
            print("The new image is predicted to be cancerous.")
            print(prediction)
        else:
            print("The new image is predicted to be non-cancerous.")
            print(prediction)

def get_accuracy_and_loss(model,X_test,y_test,X_train,y_train):
    val_loss, val_acc = model.evaluate(np.stack(X_test["image_data"]), y_test)
    print("Validation loss:", val_loss)
    print("Validation accuracy:", val_acc)
    # Evaluate on the training set
    
    train_loss, train_acc = model.evaluate(np.stack(X_train["image_data"]), y_train)
    print("Training loss:", train_loss)
    print("Training accuracy:", train_acc)

### Splitting the data into train and test sets

In [14]:
X = data.drop("isCancerous", axis=1)
X = X.drop("InstanceID",axis=1)
X = X.drop("patientID",axis=1)
y = data["isCancerous"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
display(X)


,ImageName,cellTypeName,cellType,image_data
0,22405.png,fibroblast,0,"[[[242, 213, 237], [239, 210, 234], [236, 206,..."
1,22406.png,fibroblast,0,"[[[221, 175, 211], [223, 175, 211], [229, 180,..."
2,22407.png,fibroblast,0,"[[[247, 243, 247], [248, 242, 246], [247, 240,..."
3,22408.png,fibroblast,0,"[[[243, 242, 242], [243, 241, 241], [243, 241,..."
4,22409.png,fibroblast,0,"[[[189, 130, 189], [187, 127, 185], [187, 126,..."
...,...,...,...,...
9891,1625.png,epithelial,2,"[[[171, 138, 180], [206, 175, 217], [238, 209,..."
9892,1626.png,epithelial,2,"[[[212, 173, 228], [191, 151, 212], [172, 132,..."
9893,1627.png,epithelial,2,"[[[175, 141, 193], [191, 162, 205], [207, 180,..."
9894,1628.png,epithelial,2,"[[[227, 202, 232], [188, 159, 202], [180, 146,..."


In [15]:
from sklearn.model_selection import KFold
import numpy as np

# define the number of folds for cross validation
k = 5

# create a KFold object with k folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# create empty lists to store the accuracy scores for each fold
scores = []

# loop over each fold
for train_index, test_index in kf.split(X):
    # get the training and testing data for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # preprocess the data
    image_dir = "patch_images"
    image_arrays_train = []
    for image_name in X_train["ImageName"]:
        img = cv2.imread(os.path.join(image_dir, image_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (32, 32))
        image_arrays_train.append(img)
    image_arrays_train = np.array(image_arrays_train)

    image_arrays_test = []
    for image_name in X_test["ImageName"]:
        img = cv2.imread(os.path.join(image_dir, image_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (32, 32))
        image_arrays_test.append(img)
    image_arrays_test = np.array(image_arrays_test)

    # define the model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer=Adadelta(learning_rate=1e-1), metrics=["accuracy"])

    # train the model
    model.fit(image_arrays_train, y_train, epochs=30, batch_size=32, verbose=0)

    # evaluate the model on the test data for this fold
    score = model.evaluate(image_arrays_test, y_test, verbose=0)
    scores.append(score[1])

# print the mean and standard deviation of the accuracy scores for all folds
print("Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(scores)*100, np.std(scores)*100))

Accuracy: 88.74% (+/- 1.85%)


### BASELINE RGB model 

In [16]:
# Build and train a model using the merged dataset
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=Adadelta(learning_rate=1e-1), metrics=["accuracy"])
model.fit(np.stack(X_train["image_data"]), y_train, epochs=30, batch_size=32)



Epoch 1/30
248/248 [==============================] - 5s 17ms/step - loss: 3.4774 - accuracy: 0.6807
Epoch 2/30
248/248 [==============================] - 4s 17ms/step - loss: 0.4905 - accuracy: 0.7718
Epoch 3/30
248/248 [==============================] - 4s 16ms/step - loss: 0.4330 - accuracy: 0.8190
Epoch 4/30
248/248 [==============================] - 4s 16ms/step - loss: 0.3950 - accuracy: 0.8354
Epoch 5/30
248/248 [==============================] - 4s 15ms/step - loss: 0.3682 - accuracy: 0.8484
Epoch 6/30
248/248 [==============================] - 4s 15ms/step - loss: 0.3562 - accuracy: 0.8545
Epoch 7/30
248/248 [==============================] - 4s 16ms/step - loss: 0.3412 - accuracy: 0.8588
Epoch 8/30
248/248 [==============================] - 4s 15ms/step - loss: 0.3296 - accuracy: 0.8661
Epoch 9/30
248/248 [==============================] - 4s 15ms/step - loss: 0.3365 - accuracy: 0.8669
Epoch 10/30
248/248 [==============================] - 4s 15ms/step - loss: 0.3314 - accura

In [17]:
get_accuracy_and_loss(model,X_test,y_test,X_train,y_train)
predict_extra_data(model)

62/62 [==============================] - 1s 8ms/step - loss: 0.2860 - accuracy: 0.8909
Validation loss: 0.28602665662765503
Validation accuracy: 0.8908539414405823
248/248 [==============================] - 2s 7ms/step - loss: 0.2434 - accuracy: 0.9039
Training loss: 0.24343055486679077
Training accuracy: 0.9038777351379395
1/1 [==============================] - 0s 56ms/step
The new image is predicted to be non-cancerous.
0.00023519235
1/1 [==============================] - 0s 20ms/step
The new image is predicted to be non-cancerous.
0.015302624
1/1 [==============================] - 0s 21ms/step
The new image is predicted to be non-cancerous.
0.027006261
1/1 [==============================] - 0s 21ms/step
The new image is predicted to be non-cancerous.
0.006662449
1/1 [==============================] - 0s 21ms/step
The new image is predicted to be non-cancerous.
0.010664831
1/1 [==============================] - 0s 17ms/step
The new image is predicted to be non-cancerous.
0.0001616606

In [18]:
new_image = cv2.imread("patch_images/22409.png")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (32, 32))
new_image = np.array(new_image)
new_image = np.expand_dims(new_image, axis=0)
prediction = model.predict(new_image)
if prediction >= 0.5:
    print("The new image is predicted to be cancerous.")
    print(prediction)
else:
    print("The new image is predicted to be non-cancerous.")
    print(prediction)


1/1 [==============================] - 0s 17ms/step
The new image is predicted to be non-cancerous.
[[0.00188357]]


# RGB model with Data Augmentation 

In [47]:
# Create an instance of the ImageDataGenerator class and specify the transformations to apply
datagen = ImageDataGenerator( 
    zoom_range=0.2, 
    width_shift_range=0.01, # shift the image horizontally by up to 10%
    height_shift_range=0.01, # shift the image vertically by up to 10%
    shear_range=0.02, # shear the image by up to 10%
    horizontal_flip=True, # flip the image horizontally
    fill_mode='nearest') # fill any empty pixels with the nearest value

# Fit the generator on the training data
datagen.fit(np.stack(X_train["image_data"]))

data_aug_model = Sequential()
data_aug_model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)))
data_aug_model.add(MaxPooling2D(pool_size=(2, 2)))
data_aug_model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
data_aug_model.add(MaxPooling2D(pool_size=(2, 2)))
data_aug_model.add(Flatten())
data_aug_model.add(Dense(128, activation="relu"))
data_aug_model.add(Dropout(0.5))
data_aug_model.add(Dense(1, activation="sigmoid"))
data_aug_model.compile(loss="binary_crossentropy", optimizer=RMSprop(learning_rate=1e-4), metrics=["accuracy"])

# fit the model using the generator
data_aug_model.fit(datagen.flow(np.stack(X_train["image_data"]), y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=20)


Epoch 1/20
247/247 [==============================] - 6s 20ms/step - loss: 1.1107 - accuracy: 0.7552
Epoch 2/20
247/247 [==============================] - 5s 20ms/step - loss: 0.4162 - accuracy: 0.8272
Epoch 3/20
247/247 [==============================] - 5s 21ms/step - loss: 0.3804 - accuracy: 0.8412
Epoch 4/20
247/247 [==============================] - 5s 20ms/step - loss: 0.3691 - accuracy: 0.8506
Epoch 5/20
247/247 [==============================] - 5s 20ms/step - loss: 0.3508 - accuracy: 0.8597
Epoch 6/20
247/247 [==============================] - 5s 20ms/step - loss: 0.3455 - accuracy: 0.8549
Epoch 7/20
247/247 [==============================] - 5s 20ms/step - loss: 0.3357 - accuracy: 0.8603
Epoch 8/20
247/247 [==============================] - 5s 20ms/step - loss: 0.3320 - accuracy: 0.8643
Epoch 9/20
247/247 [==============================] - 5s 20ms/step - loss: 0.3147 - accuracy: 0.8693
Epoch 10/20
247/247 [==============================] - 5s 20ms/step - loss: 0.3219 - accura

In [48]:
new_image = cv2.imread("patch_images/12681.png")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (32, 32))
new_image = np.array(new_image)
new_image = np.expand_dims(new_image, axis=0)
prediction = data_aug_model.predict(new_image)
if prediction >= 0.5:
    print("The new image is predicted to be cancerous.")
    print(prediction)
else:
    print("The new image is predicted to be non-cancerous.")
    print(prediction)

1/1 [==============================] - 0s 54ms/step
The new image is predicted to be non-cancerous.
[[0.14547583]]


In [49]:
predict_extra_data(data_aug_model)
get_accuracy_and_loss(data_aug_model,X_test,y_test,X_train,y_train)

1/1 [==============================] - 0s 17ms/step
The new image is predicted to be non-cancerous.
[[0.14547583]]
1/1 [==============================] - 0s 18ms/step
The new image is predicted to be non-cancerous.
[[0.29256076]]
1/1 [==============================] - 0s 27ms/step
The new image is predicted to be non-cancerous.
[[0.15583122]]
1/1 [==============================] - 0s 19ms/step
The new image is predicted to be non-cancerous.
[[0.1505906]]
1/1 [==============================] - 0s 20ms/step
The new image is predicted to be non-cancerous.
[[0.35594097]]
1/1 [==============================] - 0s 24ms/step
The new image is predicted to be non-cancerous.
[[0.11479353]]
1/1 [==============================] - 0s 18ms/step
The new image is predicted to be non-cancerous.
[[0.17687058]]
1/1 [==============================] - 0s 18ms/step
The new image is predicted to be non-cancerous.
[[0.41733012]]
1/1 [==============================] - 0s 18ms/step
The new image is predicted to

### RGB model with a deeper network (more layers)

In [50]:
deeper_model = Sequential()
deeper_model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)))
deeper_model.add(MaxPooling2D(pool_size=(2, 2)))
deeper_model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
deeper_model.add(MaxPooling2D(pool_size=(2, 2)))
deeper_model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
deeper_model.add(MaxPooling2D(pool_size=(2, 2)))
deeper_model.add(Flatten())
deeper_model.add(Dense(256, activation="relu"))
deeper_model.add(Dropout(0.5))
deeper_model.add(Dense(128, activation="relu"))
deeper_model.add(Dropout(0.5))
deeper_model.add(Dense(1, activation="sigmoid"))

deeper_model.compile(loss="binary_crossentropy", optimizer=RMSprop(learning_rate=1e-4), metrics=["accuracy"])

deeper_model.fit(np.stack(X_train["image_data"]), y_train, epochs=30, batch_size=32)


Epoch 1/30
248/248 [==============================] - 5s 19ms/step - loss: 1.4587 - accuracy: 0.6565
Epoch 2/30
248/248 [==============================] - 5s 19ms/step - loss: 0.4918 - accuracy: 0.7871
Epoch 3/30
248/248 [==============================] - 4s 18ms/step - loss: 0.4225 - accuracy: 0.8201
Epoch 4/30
248/248 [==============================] - 5s 20ms/step - loss: 0.3903 - accuracy: 0.8355
Epoch 5/30
248/248 [==============================] - 5s 20ms/step - loss: 0.3725 - accuracy: 0.8447
Epoch 6/30
248/248 [==============================] - 5s 20ms/step - loss: 0.3506 - accuracy: 0.8514
Epoch 7/30
248/248 [==============================] - 5s 19ms/step - loss: 0.3359 - accuracy: 0.8638
Epoch 8/30
248/248 [==============================] - 5s 19ms/step - loss: 0.3340 - accuracy: 0.8660
Epoch 9/30
248/248 [==============================] - 5s 20ms/step - loss: 0.3259 - accuracy: 0.8681
Epoch 10/30
248/248 [==============================] - 5s 20ms/step - loss: 0.3168 - accura

In [51]:
new_image = cv2.imread("patch_images/22409.png")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (32, 32))
new_image = np.array(new_image)
new_image = np.expand_dims(new_image, axis=0)
prediction = deeper_model.predict(new_image)
if prediction >= 0.5:
    print("The new image is predicted to be cancerous.")
    print(prediction)
else:
    print("The new image is predicted to be non-cancerous.")
    print(prediction)

1/1 [==============================] - 0s 62ms/step
The new image is predicted to be non-cancerous.
[[0.01378987]]


In [52]:
predict_extra_data(deeper_model)
get_accuracy_and_loss(deeper_model,X_test,y_test,X_train,y_train)

1/1 [==============================] - 0s 16ms/step
The new image is predicted to be non-cancerous.
[[4.120697e-05]]
1/1 [==============================] - 0s 20ms/step
The new image is predicted to be non-cancerous.
[[0.24716307]]
1/1 [==============================] - 0s 26ms/step
The new image is predicted to be non-cancerous.
[[0.18089657]]
1/1 [==============================] - 0s 21ms/step
The new image is predicted to be non-cancerous.
[[0.02440446]]
1/1 [==============================] - 0s 17ms/step
The new image is predicted to be non-cancerous.
[[0.037422]]
1/1 [==============================] - 0s 17ms/step
The new image is predicted to be non-cancerous.
[[0.03327738]]
1/1 [==============================] - 0s 17ms/step
The new image is predicted to be non-cancerous.
[[0.03030089]]
1/1 [==============================] - 0s 19ms/step
The new image is predicted to be non-cancerous.
[[0.07249901]]
1/1 [==============================] - 0s 18ms/step
The new image is predicted t

In [54]:
csv_file = "data_labels_mainData.csv"
data = pd.read_csv(csv_file)

# Step 2: Read the images from the directory and convert them to arrays
image_dir = "patch_images"
image_arrays = []
for image_name in data["ImageName"]:
    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (32, 32)) / 255.0
    image_arrays.append(img)
image_arrays = np.array(image_arrays)

# Step 3: Merge the image arrays with the CSV dataframe
data["image_data"] = list(image_arrays)
data = data.drop("ImageName", axis=1)

# Step 4: Split the data into training and testing sets
X = data.drop("isCancerous", axis=1)
y = data["isCancerous"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create an ImageDataGenerator instance for data augmentation
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, vertical_flip=True)

test_datagen = ImageDataGenerator()

# Step 6: Create a generator for training data using the ImageDataGenerator instance
train_generator = train_datagen.flow(np.stack(X_train["image_data"]), y_train, batch_size=32)

# Step 7: Create a generator for validation data using the ImageDataGenerator instance
valid_generator = test_datagen.flow(np.stack(X_test["image_data"]), y_test, batch_size=32)

# Step 8: Build and train a model using the merged dataset and data augmentation
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten and add dense layers
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(loss="binary_crossentropy", optimizer=RMSprop(learning_rate=1e-4), metrics=["accuracy"])

# Fit the model using data augmentation
model.fit(train_generator, epochs=100, validation_data=valid_generator)


Epoch 1/100
248/248 [==============================] - 43s 166ms/step - loss: 0.5835 - accuracy: 0.7494 - val_loss: 2.6387 - val_accuracy: 0.3995
Epoch 2/100
248/248 [==============================] - 38s 152ms/step - loss: 0.4410 - accuracy: 0.8321 - val_loss: 2.2389 - val_accuracy: 0.4045
Epoch 3/100
248/248 [==============================] - 37s 151ms/step - loss: 0.4068 - accuracy: 0.8430 - val_loss: 1.7325 - val_accuracy: 0.5359
Epoch 4/100
248/248 [==============================] - 37s 151ms/step - loss: 0.3754 - accuracy: 0.8535 - val_loss: 2.4103 - val_accuracy: 0.4480
Epoch 5/100
248/248 [==============================] - 37s 151ms/step - loss: 0.3603 - accuracy: 0.8586 - val_loss: 0.9431 - val_accuracy: 0.6768
Epoch 6/100
248/248 [==============================] - 37s 151ms/step - loss: 0.3534 - accuracy: 0.8576 - val_loss: 0.7896 - val_accuracy: 0.7379
Epoch 7/100
248/248 [==============================] - 38s 151ms/step - loss: 0.3257 - accuracy: 0.8661 - val_loss: 0.3710 -

In [64]:
new_image = cv2.imread("patch_images/22409.png")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (32, 32))
new_image = np.array(new_image)
new_image = np.expand_dims(new_image, axis=0)
prediction = model.predict(new_image)[0][0]
if prediction >= 0.5:
    print("The new image is predicted to be cancerous.")
    print(prediction)
else:
    print("The new image is predicted to be non-cancerous.")
    print(prediction)

1/1 [==============================] - 0s 22ms/step
The new image is predicted to be non-cancerous.
0.0


In [65]:
predict_extra_data(model)
get_accuracy_and_loss(model,X_test,y_test,X_train,y_train)

1/1 [==============================] - 0s 22ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 26ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 25ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 21ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 22ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 23ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 21ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 24ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 29ms/step
The new image is predicted to be non-cancerous.
0.0
1/1 [==============================] - 0s 25ms/step
The new imag

## Using grayscale

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

# Step 1: Read the CSV file and create a dataframe
csv_file = "data_labels_mainData.csv"
data = pd.read_csv(csv_file)

# Step 2: Read the images from the directory and convert them to arrays
image_dir = "patch_images"
image_arrays = []
for image_name in data["ImageName"]:
    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale format
    img = cv2.resize(img, (224, 224))
    image_arrays.append(img)
image_arrays = np.array(image_arrays)

In [ ]:
data["image_data"] = list(image_arrays)
data = data.drop("ImageName", axis=1)

# Step 4: Split the data into training and testing sets
X = data.drop("isCancerous", axis=1)
y = data["isCancerous"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 5: Define the model
def create_model(conv1_filters=32, conv2_filters=64, dense_units=128, dropout_rate=0.5):
    model = Sequential()
    model.add(Conv2D(conv1_filters, kernel_size=(3, 3), activation="relu", input_shape=(224, 224, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(conv2_filters, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(dense_units, activation="relu"))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


# Step 6: Define the hyperparameters to tune and perform the search
param_grid = {
    "conv1_filters": [32, 64],
    "conv2_filters": [64, 128],
    "dense_units": [128, 256],
    "dropout_rate": [0.3, 0.5],
}
deeper_model = KerasClassifier(
    build_fn=create_model,
    epochs=10,
    batch_size=32,
    verbose=2
)
grid_search = GridSearchCV(
    estimator=deeper_model,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring="accuracy",
)
grid_search.fit(np.stack(X_train["image_data"]), y_train)

# Step 7: Print the best parameters and accuracy score
print("Best parameters:", grid_search.best_params_)
print("Accuracy score:", grid_search.best_score_)

In [ ]:

# Step 5: Make predictions on a new image
new_image_path = "patch_images/4850.png"
new_image = cv2.resize(cv2.imread(new_image_path, cv2.IMREAD_GRAYSCALE), (224, 224)) / 255.0
new_image_array = np.array([new_image])
prediction = grid_search.predict(new_image_array)
if prediction[0] > 0.5:
    print("The image is cancerous.")
else:
    print("The image is not cancerous.")


In [ ]:
y_pred = grid_search.predict(np.stack(X_test["image_data"]))
y_pred = (y_pred > 0.5).astype(int)

# Step 7: Evaluate the model's performance on the test data
accuracy = (y_pred == y_test.values.reshape(-1,1)).mean()
print("Accuracy on test set:", accuracy)